# Showcase of Bybit Future - Future calc
The BybitClient has custom methods that I coded, but can also make normal request to the Bybit API with the session attribute.

In [1]:
# Inside of the module, we use absolute imports
# Because this context is different, we need to add inside the PYTHONPATH the path of the module
import sys
sys.path.append('Bybit')

from Bybit.client import BybitClient
import Bybit.utils


# Create a Bybit client
client = BybitClient(demo=False, verbose=1)

In [2]:
# Get all futures BTC gaps
DataWrangler = client.all_gaps_pd(inverse=False, perpetual=True, pretty=True, applyFees=True)
DataWrangler

,Buy,Sell,Gap,Coeff,APR,DaysLeft,CumVolume
0,BTCPERP,BTCUSDT,$ -31.90,0.00 %,-0.00 %,-20054,7.94B
1,BTCPERP,BTC-29NOV24,$ -76.30,0.00 %,0.00 %,1,23.79M
2,BTCPERP,BTC-06DEC24,$ 257.70,0.05 %,2.14 %,8,23.15M
3,BTCPERP,BTC-13DEC24,$ 632.70,0.44 %,10.47 %,15,23.35M
4,BTCPERP,BTC-27DEC24,$ 1268.70,1.10 %,13.73 %,29,24.18M
5,BTCPERP,BTC-31JAN25,$ 2706.20,2.61 %,14.78 %,64,23.11M
6,BTCPERP,BTC-28MAR25,$ 4653.20,4.64 %,14.07 %,120,23.78M
7,BTCPERP,BTC-27JUN25,$ 7679.20,7.80 %,13.47 %,211,23.90M
8,BTCPERP,BTC-26SEP25,$ 10729.20,10.98 %,13.26 %,302,23.85M
9,BTCUSDT,BTC-29NOV24,$ -44.40,0.00 %,0.00 %,1,7.91B


In [ ]:
client.fetcher.get_USDC_BTC()
# BTCUSDT	BTC-29NOV24

await client.fetcher.exit_both_position('BTCUSDT', 'BTC-29NOV24', 0.001, 0.001)


## Tester
Just to run some tests

In [ ]:
# 1 - Simple enter position logic (given a future and a perpetual)
# 2 - Simple exit position logic (given a future and a perpetual). Can also exit when reaching a satisfying unexpected gap
# 3 - Logic to activate both operations (given a future and a perpetual). Enter again when reaching a satisfying unexpected gap
# 4 - Logic to find the best future and perpetual to trade
# 5 - Logic to keep the loop running
# 6 - Monitoring features
# 7 - Use systemctl to launch 24/7
# We use sockets, once we selected the best local future, compared to their average gap.
# Shared state to store the latest updates

2024-11-29 20:53:02,482 - Bybit.client.fetcher - INFO - Loaded 8029 existing data points.


2024-11-29 20:53:02,665 - Bybit.client.fetcher - INFO - Fetched 36 new data points.
2024-11-29 20:53:02,735 - Bybit.client.fetcher - INFO - Loaded 24087 existing data points.
2024-11-29 20:53:02,908 - Bybit.client.fetcher - INFO - Fetched 106 new data points.
2024-11-29 20:53:03,164 - Bybit.client.fetcher - INFO - Loaded 120431 existing data points.
2024-11-29 20:53:03,341 - Bybit.client.fetcher - INFO - Fetched 530 new data points.


,startTime,openPrice,highPrice,lowPrice,closePrice,volume,turnover
0,1732867140000,95818,95818,95736.5,95771,0.147,14076.8875
1,1732867080000,95817.5,95818,95817.5,95818,0.04,3832.7005
2,1732867020000,95818,95818,95817.5,95817.5,0.045,4311.7875
3,1732866960000,95785.5,95818,95785.5,95818,0.244,23377.4915
4,1732866900000,95821,95824.5,95757,95785.5,0.23,22033.7095
...,...,...,...,...,...,...,...
120955,1725609840000,56642.5,56663,56642.5,56662,0.26,14732.274
120956,1725609780000,56616.5,56642.5,56616.5,56642.5,0.73,41346.7425
120957,1725609720000,56767.5,56767.5,56616.5,56616.5,6.907,391453.3225
120958,1725609660000,56767.5,56767.5,56767.5,56767.5,0,0


## Fetching time :)

Getting 15, 5 and 1 minute candles for everything since 01/01/2024

In [2]:
# Future contracts
allContracts = client.fetcher.get_futureNames()

for contract in allContracts:
    client.fetcher.get_history_pd(contract, interval="15", dest="store", dateLimit="2024-01-01 00:00")
    client.fetcher.get_history_pd(contract, interval="5", dest="store", dateLimit="2024-01-01 00:00")
    client.fetcher.get_history_pd(contract, interval="1", dest="store", dateLimit="2024-01-01 00:00")

# spot
# client.fetcher.get_history_pd("BTCUSDC", interval="15", dateLimit="2024-01-01 00:00", category="spot", dest="store")
client.fetcher.get_history_pd("BTCUSDT", interval="15", dateLimit="2024-01-01 00:00", category="spot", dest="store")
client.fetcher.get_history_pd("BTCUSDT", interval="5", dateLimit="2024-01-01 00:00", category="spot", dest="store")
client.fetcher.get_history_pd("BTCUSDT", interval="1", dateLimit="2024-01-01 00:00", category="spot", dest="store")

# Perpetual contracts
client.fetcher.get_history_pd("BTCUSDT", interval="15", dateLimit="2024-01-01 00:00", dest="store")
client.fetcher.get_history_pd("BTCUSDT", interval="5", dateLimit="2024-01-01 00:00", dest="store")
client.fetcher.get_history_pd("BTCUSDT", interval="1", dateLimit="2024-01-01 00:00", dest="store")


2024-11-29 20:51:07,192 - Bybit.client.fetcher - INFO - Loaded 1309 existing data points.
2024-11-29 20:51:07,376 - Bybit.client.fetcher - INFO - Fetched 88 new data points.
2024-11-29 20:51:07,503 - Bybit.client.fetcher - INFO - Loaded 3927 existing data points.
2024-11-29 20:51:07,678 - Bybit.client.fetcher - INFO - Fetched 261 new data points.
2024-11-29 20:51:07,777 - Bybit.client.fetcher - INFO - Loaded 19631 existing data points.
2024-11-29 20:51:07,953 - Bybit.client.fetcher - INFO - Fetched 1000 new data points.
2024-11-29 20:51:08,138 - Bybit.client.fetcher - INFO - Fetched 303 new data points.
2024-11-29 20:51:08,172 - Bybit.client.fetcher - INFO - Loaded 637 existing data points.
2024-11-29 20:51:08,344 - Bybit.client.fetcher - INFO - Fetched 88 new data points.
2024-11-29 20:51:08,397 - Bybit.client.fetcher - INFO - Loaded 1911 existing data points.
2024-11-29 20:51:08,584 - Bybit.client.fetcher - INFO - Fetched 261 new data points.
2024-11-29 20:51:08,656 - Bybit.client.fe

,startTime,openPrice,highPrice,lowPrice,closePrice,volume,turnover
0,1732895400000,98250,98313.6,98169.7,98297.8,88.039,8650764.0576
1,1732895340000,98181.7,98250,98142.3,98250,49.258,4837289.0139
2,1732895280000,98197,98237.7,98124.3,98181.7,66.335,6512087.6629
3,1732895220000,98174.5,98241.4,98125.7,98197,90.901,8924385.1906
4,1732895160000,98179.7,98251,98073.9,98174.5,211.628,20770011.6268
...,...,...,...,...,...,...,...
481239,1704021060000,42653.6,42653.6,42637.8,42648.1,21.071,898582.7175
481240,1704021000000,42661.7,42670,42653.5,42653.6,41.295,1761664.4116
481241,1704020940000,42647.4,42661.7,42647.3,42661.7,40.159,1712828.847
481242,1704020880000,42697.5,42697.6,42633.3,42647.4,210.474,8977606.5368


## Plot testing

Part to test the plot of the data.


In [ ]:
# Yes

## Long Spot x Short Future

Spot USDC is 0% fees and Future is 0.0550% taker fees (0% with contract delivery).
Problem is that spot cannot be used with leverage. Could borrow but that would be awkward. (tried it)

In [ ]:
Bybit.utils.load_klines_parquet("store/BTCUSDT_1.parquet", pretty=True)

In [ ]:
# Long position, then short position
Bybit.utils.plot_compare("store/BTCUSDT_1.parquet", "store/BTC-29NOV24_1.parquet", dateLimit="2024-11-24 00:00")

## Long Perpetual x Short Future

Perpetual USDT is 0.0550% taker fees and Future is 0.0550% taker fees (0% with contract delivery).
Yes we have fees, but wen can use leverage on both sides.

We use Perpetual USDT because there is more liquidity.

In [ ]:
Bybit.utils.load_klines_parquet("store/BTCUSDT_15.parquet", pretty=True)

In [ ]:
Bybit.utils.plot_compare("store/BTCUSDT_15.parquet", "store/BTC-29NOV24_15.parquet")

## Long Spot x Short Inverse Future

What if I buy spot and collateralize it to short inverse future?
We effectively capture the difference.

In [ ]:
# Keep going when get_history_pd file names are changed

## OKX 20x Leverage test

I launched a test with OKX with a short on BTCUSDT-27JUN25 and long on BTCUSDT-29NOV24.

Following section will track the the two contracts' data

In [ ]:
Bybit.utils.plot_compare("store/BTC-29NOV24_15.parquet", "store/BTC-27JUN25_15.parquet", dateLimit="2024-05-06")